In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words=open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars=sorted(list(set("".join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}

In [6]:
#Build the dataser
block_size=3
X,Y=[],[]
for w in words[:5]:
    print(w)
    context=[0]*block_size
    for ch in w +'.':
        ix=stoi[ch]
        X.append(context)
        Y.append(ix)
        print("".join(itos[i] for i in context),'-->',itos[ix])
        context=context[1:]+[ix]
X=torch.tensor(X)
Y=torch.tensor(Y)

emma
... --> e
..e --> m
.em --> m
emm --> a
mma --> .
olivia
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
ava
... --> a
..a --> v
.av --> a
ava --> .
isabella
... --> i
..i --> s
.is --> a
isa --> b
sab --> e
abe --> l
bel --> l
ell --> a
lla --> .
sophia
... --> s
..s --> o
.so --> p
sop --> h
oph --> i
phi --> a
hia --> .


In [7]:
X.shape,X.dtype,Y.shape,Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [9]:
C=torch.randn((27,2))
C

tensor([[ 1.3510, -1.1729],
        [ 0.9085,  0.7254],
        [-0.1368,  0.5607],
        [-0.6262, -0.9350],
        [ 0.1124,  0.2941],
        [-0.7770, -0.2515],
        [ 0.5807,  1.1522],
        [-1.0065, -0.3359],
        [-0.1485, -1.3269],
        [-0.1525,  0.4478],
        [-0.2407, -1.3738],
        [-0.6895,  0.1926],
        [ 1.0237, -0.6837],
        [-0.6218, -0.9304],
        [ 1.0014,  2.7420],
        [ 0.3873,  0.2464],
        [-0.0287, -0.2944],
        [ 1.7513, -0.6085],
        [-0.9231, -1.5497],
        [ 0.9042, -0.2828],
        [-0.6517,  0.1680],
        [ 1.2577,  0.0889],
        [-0.4417,  1.4701],
        [-0.3856, -0.8597],
        [ 0.0478, -0.0362],
        [-1.0189, -1.7198],
        [-0.1046, -0.4144]])

In [11]:
emb=C[X]

In [15]:
W1=torch.randn((6,100))
b1=torch.randn(100)

In [ ]:
h=torch.tanh((emb.view(-1,6) @ W1) + b1)
h.shape

torch.Size([32, 100])